<b> Lab 3 | Week 2 Day 3 </b> 

### Multi-Agent Multi-Model system for generating and sending cold sales emails.!!

Build a Multi-Agent Multi Model cold sales outreach emails system:
1. Use of Different Models from different Labs
2. Structured Outputs
3. Guardrails

In [ ]:
import os, sendgrid
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:

openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set")

### Multi-model setup for the three sales agents

In [ ]:
# This is the code that Ed used in his lab by directly accessing the OpenAI compatible endpoints of Gemini, Deepseek and GroQ

"""
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
"""

Here we leverage openrouter to achieve the same (instead of the respective direct access):
- Set the openrouter base URL
- Setup and point the OpenAI client at OpenRouter’s OpenAI-compatible endpoint by passing in the base URL and the openrouter api key
- Define three individual model wrappers by passing the model speification (provider/model name) along with the openrouter client name
- These three models will be used by the three sales agents

In [ ]:
# 1) Define OpenRouter’s OpenAI-compatible endpoint
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

In [ ]:
# 2) create an AsyncOpenAI client that talks to OpenRouter
openrouter_client = AsyncOpenAI(
    base_url=OPENROUTER_BASE_URL, 
    api_key=openrouter_api_key
    )

# 2) create the model wrapper using the exact OpenRouter model id
claude_model = OpenAIChatCompletionsModel(model="anthropic/claude-3-7-sonnet", openai_client=openrouter_client)
gemini_model = OpenAIChatCompletionsModel(model="google/gemini-2.5-flash", openai_client=openrouter_client)
deepseek_model = OpenAIChatCompletionsModel(model="deepseek/deepseek-chat", openai_client=openrouter_client)

In [ ]:
# Let's define the System Prompt/Instructions for the 3 sales agents

# Instructions for the professional agent
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# Instructions for the engaging agent
instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# Instructions for the busy agent
instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
# 4) create the agent as before
sales_agent1 = Agent(name="Claude Sales Agent", instructions=instructions1, model=claude_model)
sales_agent2 = Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3 = Agent(name="DeepSeek Sales Agent", instructions=instructions3, model=deepseek_model)

In [ ]:
# Convert the agents as tools
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

Setup the Emailer Agent

In [ ]:
# Setup the Function Tool to send HTML email

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))

    from_email = Email("skillshift.ai@gmail.com") # SendGrid verified sender
    to_email = To("melbith@gmail.com")
    content = Content("text/html", html_body)

    mail = Mail(from_email, to_email, subject, content)

    try:
        sg.client.mail.send.post(request_body=mail.get())
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
# Setup the Subject Writer and HTML Converter Agents

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-5-mini")
html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-5-mini")

subject_writer_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")
html_converter_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
# Assemble the tools for the Emailer Agent
emailer_tools = [subject_writer_tool, html_converter_tool, send_html_email]

In [ ]:
# Finally, define the Emailer Agent

emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=emailer_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
# Define the tools and handoffs for the top level Sales Manager

sales_manager_tools = [tool1, tool2, tool3] # 3 sales agents as tools
sales_manager_handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your job is to generate cold sales emails using the tools available to you.
You must never write or modify sales emails yourself — all email drafts must come from the sales_agent tools. Just ask them to write the email.

You must call all three sales_agent tools at least once because each produces a different style of email. If any draft is weak or unclear, 
you may call the same tool again to request another version but limit to a maximum of three attempts.

Use your judgment to choose the single best email — the one most likely to get a reply from the prospect.

After selecting the winning email, perform a handoff to the Email Manager agent.
You must hand off exactly one email and no more.
"""

In [ ]:
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=sales_manager_tools,
    handoffs=sales_manager_handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR-Multi Model"):
    result = await Runner.run(sales_manager, message)

Checkout the trace here: https://platform.openai.com/traces and the email sent by Sendgrid

---

We will explore Guardrails and Structured output

- We set agents to act as Guardrails checking the intended behaviour
- Guardrails can be applied to the input of the first agent or the output of the last agent; NOT in between

In [ ]:
# Define the output type for the guardrail agent using Pydantic object

class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# Define the guardrail agent
guardrail_agent = Agent(
    name="Name Checker",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    model="gpt-4o-mini",
    output_type=NameCheckOutput
)


In [ ]:

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context) # Run the guardrail agent

    is_name_in_message = result.final_output.is_name_in_message # Check if the name is in the message
    found_name = result.final_output # Get the name from the guardrail agent

    return GuardrailFunctionOutput(
        output_info={"found_name": found_name},
        tripwire_triggered=is_name_in_message
    )

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=sales_manager_tools,
    handoffs=sales_manager_handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR-Multi Model"):
    result = await Runner.run(careful_sales_manager, message)

Checkout the trace here: https://platform.openai.com/traces and the email sent by Sendgrid

In [ ]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR-Multi Model"):
    result = await Runner.run(careful_sales_manager, message)